In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls ./drive/MyDrive/CS5242_project/personal_testing/data

CAM16_100cls_10mask  WBC_1  WBC_10  WBC_100  WBC_50


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from tqdm import tqdm

# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the WBC dataset
#data_dir = "./data/WBC_1/train/data"
data_dir = "./drive/MyDrive/CS5242_project/personal_testing/data/WBC_1/train/data"
wbc_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
wbc_loader = DataLoader(wbc_dataset, batch_size=32, shuffle=True)

# Define the ResNet-18 model without pretraining
model = models.resnet18(pretrained=False)
num_classes = len(wbc_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set up criterion (loss function) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # You can adjust the number of epochs as needed
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in tqdm(wbc_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    average_loss = total_loss / len(wbc_loader)
    accuracy = (correct_predictions / total_samples) * 100
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')

print("Training finished!")

# Define transformations for the evaluation dataset (without shuffling)
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the WBC evaluation dataset
eval_data_dir = "./drive/MyDrive/CS5242_project/personal_testing/data/WBC_100/val/data"
#eval_data_dir = "./data/WBC_100/val/data"
eval_dataset = datasets.ImageFolder(root=eval_data_dir, transform=eval_transform)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

# ...

# Calculate final accuracy on the evaluation dataset
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in tqdm(eval_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

final_accuracy = (total_correct / total_samples) * 100
print(f'Final Accuracy on the evaluation dataset: {final_accuracy:.2f}%')


Epoch 1/10: 100%|██████████| 3/3 [00:00<00:00,  3.67it/s]


Epoch [1/10], Loss: 1.1114, Accuracy: 52.44%


Epoch 2/10: 100%|██████████| 3/3 [00:00<00:00,  3.93it/s]


Epoch [2/10], Loss: 0.8644, Accuracy: 68.29%


Epoch 3/10: 100%|██████████| 3/3 [00:00<00:00,  3.92it/s]


...

Epoch 9/10: 100%|██████████| 3/3 [00:00<00:00,  3.94it/s]


Epoch [9/10], Loss: 0.1204, Accuracy: 96.34%


Epoch 10/10: 100%|██████████| 3/3 [00:00<00:00,  4.05it/s]


Epoch [10/10], Loss: 0.0400, Accuracy: 98.78%
Training finished!


Evaluating:  61%|██████    | 33/54 [05:00<03:43, 10.63s/it]